## For only 1 class

In [17]:
import cv2
import albumentations as albu
import torch
import segmentation_models_pytorch as smp
# doesn't matter for inference
ENCODER = 'resnet101'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['coal']
ACTIVATION = 'sigmoid'

if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'

model = smp.FPN(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    classes=len(CLASSES),
    activation=ACTIVATION,
)
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)


In [19]:
path = "E:\\few_data\\few_data\\test\\"
image_path = 'vlcsnap-2022-01-26-cut.jpg'

best_model = torch.load(r"E:\few_data\few_data\ex\best_model3.pth")
best_model.eval()

image = cv2.imread(path+image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
orig = image.copy().astype('float32')

trf = albu.Compose([albu.Lambda(image=preprocessing_fn)])
image = trf(image=image)['image']
image = cv2.resize(image,(512,512))
image = image.transpose(2, 0, 1).astype('float32')

x_tensor = torch.from_numpy(image).to('cuda').unsqueeze(0)
pr_mask = best_model.predict(x_tensor)

pr_mask1 = ((pr_mask.squeeze(0).cpu().numpy())*255).transpose(1, 2, 0)
pr_mask1 = cv2.resize(pr_mask1,(1120,512))
cv2.imwrite(path+"mask_"+image_path, pr_mask1)

test = pr_mask.squeeze(0).cpu().numpy()
test = test.transpose(1, 2,0)
test = cv2.cvtColor(test, cv2.COLOR_GRAY2RGB)
test = cv2.resize(test,(1120,512))
test = 1.0 - test

foreground = cv2.multiply(orig, test)

cv2.imwrite(r"E:\few_data\few_data\test\predicted_res1.jpg",foreground)

True